In [10]:
import os
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from performance_tests.cfsm_incrementer import CFSMIncrementer
from performance_tests.resources.cfsm import cfsm as initial_cfsm, initial_index


incrementer = CFSMIncrementer()
index = initial_index
points_number = 12
randomize_times = 10
path = f'results/{points_number}_points_{randomize_times}_times'
if not os.path.exists(path):
    os.makedirs(path)

In [11]:
import csv

def write_in_csv(filename, data):
    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(data)
        
def create_csv_result(testname):
    write_in_csv(f'{path}/{testname}.csv', ['size', 'avg_time'])

def write_line_result(testname, data):
    write_in_csv(f'{path}/{testname}.csv', data)
    
create_csv_result('simple_increment')
create_csv_result('increment_splitting_transitions')
create_csv_result('increment_splitting_transitions_in_new_states')

In [12]:
def size_of_cfsm(cfsm):
    states = len(cfsm.states)
    transitions = sum(len(transitions) for transitions in cfsm.transitions_by_source_id.values())
    return states * transitions

In [13]:
def calculate_bisimulation_time_for(cfsm, randomize_times):
    times = []
    for i in range(randomize_times):
        print(f'  {i+1} randomize')
        randomized_cfsm = incrementer.randomize(cfsm)

        start_time = time.time()
        result = cfsm.calculate_bisimulation_with(randomized_cfsm)
        end_time = time.time()
        print(f'{end_time - start_time} seconds')

        times.append(end_time - start_time)

    return np.mean(times)

In [14]:
def get_bisimulation_metrics_using(cfsm, points_number, randomize_times, increment_strategy, testname):
    metrics = []
    for i in range(points_number):
        if i != 0:
            cfsm = increment_strategy(cfsm)

        size = size_of_cfsm(cfsm)
        print(f'Calculating: {i+1} metric, size: {size}')
        avg_time = calculate_bisimulation_time_for(cfsm, randomize_times)
        metric = {'size': size, 'avg_time': avg_time}
        print(metric)
        print('')
        metrics.append(metric)
        write_line_result(testname, [size, avg_time])

    return metrics

---
## Simple increment strategy

In [9]:
def simple_increment_strategy(cfsm):
    global simple_increment_index
    global simple_increment_size
    new_cfsm, new_index = incrementer.increment(cfsm, simple_increment_size, simple_increment_index)
    cfsm_array.append(new_cfsm)
    simple_increment_index = new_index
    return new_cfsm

In [ ]:
cfsm_array = []
simple_increment_index = initial_index
simple_increment_size = 2
testname = 'simple_increment'

simple_metrics = get_bisimulation_metrics_using(
    initial_cfsm,
    points_number=points_number,
    randomize_times=randomize_times,
    increment_strategy=simple_increment_strategy,
    testname=testname
)

testname---
## Increment Splitting transitions

In [15]:
def increment_splitting_transitions_strategy(cfsm):
    global splitting_transitions_index
    global splitting_transitions_size
    cfsm, new_index = incrementer.increment_splitting_transitions(cfsm, splitting_transitions_size, splitting_transitions_index)
    splitting_transitions_index = new_index
    return cfsm

In [ ]:
splitting_transitions_index = initial_index
splitting_transitions_size = 4
testname = 'increment_splitting_transitions'

splitting_transitions_metrics = get_bisimulation_metrics_using(
    initial_cfsm,
    points_number=points_number,
    randomize_times=randomize_times,
    increment_strategy=increment_splitting_transitions_strategy,
    testname=testname
)

---
## Increment Splitting transitions in new states strategy

In [ ]:
def increment_splitting_transitions_in_new_states_strategy(cfsm):
    global splitting_transitions_in_new_states_index
    global splitting_transitions_in_new_states_size
    cfsm, new_index = incrementer.increment_splitting_transitions(cfsm, splitting_transitions_in_new_states_size, splitting_transitions_in_new_states_index)
    splitting_transitions_in_new_states_index = new_index
    return cfsm

In [ ]:
splitting_transitions_in_new_states_index = initial_index
splitting_transitions_in_new_states_size = 2
testname = 'increment_splitting_transitions_in_new_states'

splitting_transitions_metrics = get_bisimulation_metrics_using(
    initial_cfsm,
    points_number=points_number,
    randomize_times=randomize_times,
    increment_strategy=increment_splitting_transitions_in_new_states_strategy,
    testname=testname
)